In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import sys
import unicodedata
import string
import operator
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy import sparse
from scipy.spatial.distance import cosine
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
stdout = sys.stdout
reload(sys)  
sys.setdefaultencoding('utf8')
sys.stdout = stdout

In [2]:
import jsonlines
import numpy as np
import cPickle as pickle
import os

class Preprocessor:
    def __init__(self, dataDir = './'):
        self.data = []
        self.dataDir = dataDir 

    def isSampleValid(self, sample):
        return sample['gold_label'] != '-'
    
    def getDataFromJSONL(self, filename):
        numInvalid = 0
        with open(filename) as fp:
            reader = jsonlines.Reader(fp)
            for obj in reader.iter(type=dict, skip_invalid=True):
                if self.isSampleValid(obj):
                    sample = {}
                    sample['gold_label'] = obj['gold_label']
                    sample['sentence1'] = obj['sentence1']
                    sample['sentence2'] = obj['sentence2']
                    self.data.append(sample)
                else:
                    numInvalid+=1
        print len(self.data)
        print numInvalid
    
    

In [3]:
pp = Preprocessor()
pp.getDataFromJSONL('./snli_1.0_train.jsonl')

549367
785


In [4]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
BLEU_Score_List = []
for j in range(len(pp.data)):
    weights = [1./4., 1./4., 1./4., 1./4.]
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    reference = [[i for i in sentence1.lower().split() if i not in stop]]
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    candidate = [i for i in sentence2.lower().split() if i not in stop]
    length = min([len(reference[0]), len(candidate)]) 
    if length == 0:
        BLEU_Score_List.append(0)
    else:
        if length < 4:
            weights = ( 1. / length ,) * length
        score = sentence_bleu(reference, candidate, weights)
        BLEU_Score_List.append(score)      

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/seasnake/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
%store BLEU_Score_List > BLEU_Score_List.txt

Writing 'BLEU_Score_List' (list) to file 'BLEU_Score_List.txt'.


In [4]:
BLEU_Score_List = eval(open("BLEU_Score_List.txt").read())  
BLEU_Score_Array = np.array(BLEU_Score_List)

In [5]:
BLEU_Score_Array = BLEU_Score_Array[:, np.newaxis]

In [8]:
Target_List = [pp.data[j]['gold_label'] for j in range(len(pp.data))]

In [9]:
%store Target_List > Target_List.txt

Writing 'Target_List' (list) to file 'Target_List.txt'.


In [6]:
Target_List = eval(open("Target_List.txt").read())  
Target_Array = np.array(Target_List)

In [7]:
print Target_Array.shape

(549367,)


In [8]:
print BLEU_Score_Array.shape

(549367, 1)


In [9]:
tt = Preprocessor()
tt.getDataFromJSONL('./snli_1.0_test.jsonl')

9824
176


In [42]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
BLEU_Score_List_Test = []
for j in range(len(tt.data)):
    weights = [1./4., 1./4., 1./4., 1./4.]
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    reference = [[i for i in sentence1.lower().split() if i not in stop]]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    candidate = [i for i in sentence2.lower().split() if i not in stop]
    length = min([len(reference[0]), len(candidate)]) 
    if length == 0:
        BLEU_Score_List_Test.append(0)
    else:
        if length < 4:
            weights = ( 1. / length ,) * length
        score = sentence_bleu(reference, candidate, weights)
        BLEU_Score_List_Test.append(score)      

In [44]:
%store BLEU_Score_List_Test > BLEU_Score_List_Test.txt

Writing 'BLEU_Score_List_Test' (list) to file 'BLEU_Score_List_Test.txt'.


In [76]:
BLEU_Score_List_Test = eval(open("BLEU_Score_List_Test.txt").read())  
BLEU_Score_Array_Test = np.array(BLEU_Score_List_Test)

In [77]:
BLEU_Score_Array_Test = BLEU_Score_Array_Test[:, np.newaxis]

In [47]:
Target_List_Test = [tt.data[j]['gold_label'] for j in range(len(tt.data))]

In [48]:
%store Target_List_Test > Target_List_Test.txt

Writing 'Target_List_Test' (list) to file 'Target_List_Test.txt'.


In [12]:
Target_List_Test = eval(open("Target_List_Test.txt").read())
Target_Array_Test = np.array(Target_List_Test)

In [13]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list = y_pred.tolist()
%store y_pred_list > y_pred_list.txt

Writing 'y_pred_list' (list) to file 'y_pred_list.txt'.


In [14]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred).sum()))

Number of mislabeled points out of a total 9824 points : 5957


In [51]:
print 9824 - 5957

3867


In [52]:
print 3867./9824.

0.393627850163


In [15]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
y_pred_mnb = mnb.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list_mnb = y_pred_mnb.tolist()
%store y_pred_list_mnb > y_pred_list_mnb.txt

Writing 'y_pred_list_mnb' (list) to file 'y_pred_list_mnb.txt'.


In [16]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred_mnb).sum()))

Number of mislabeled points out of a total 9824 points : 6456


In [27]:
print 9824 - 6456

3368


In [28]:
print 3368./9824.

0.342833876221


In [57]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_svm = clf.fit(BLEU_Score_Array, Target_Array).predict(BLEU_Score_Array_Test)
y_pred_list_svm = y_pred_svm.tolist()
%store y_pred_list_svm > y_pred_list_svm.txt

Writing 'y_pred_list_svm' (list) to file 'y_pred_list_svm.txt'.


In [58]:
print("Number of mislabeled points out of a total %d points : %d" % (BLEU_Score_Array_Test.shape[0],(Target_Array_Test != y_pred_svm).sum()))

Number of mislabeled points out of a total 9824 points : 6587


In [59]:
print 9824 - 6587

3237


In [60]:
print 3237./9824.

0.329499185668


In [61]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred))

             precision    recall  f1-score   support

contradiction       0.38      0.44      0.41      3237
 entailment       0.39      0.35      0.37      3368
    neutral       0.41      0.39      0.40      3219

avg / total       0.39      0.39      0.39      9824



In [62]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_mnb))

             precision    recall  f1-score   support

contradiction       0.00      0.00      0.00      3237
 entailment       0.34      1.00      0.51      3368
    neutral       0.00      0.00      0.00      3219

avg / total       0.12      0.34      0.18      9824



In [63]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_svm))

             precision    recall  f1-score   support

contradiction       0.33      1.00      0.50      3237
 entailment       0.00      0.00      0.00      3368
    neutral       0.00      0.00      0.00      3219

avg / total       0.11      0.33      0.16      9824



In [64]:
metrics.confusion_matrix(Target_Array_Test, y_pred)


array([[1434,  919,  884],
       [1253, 1192,  923],
       [1062,  916, 1241]])

In [68]:
Length_Difference_List = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [i for i in sentence1.lower().split()]
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [i for i in sentence2.lower().split()]
    length_diff = len(premise)-len(hypothesis)
    Length_Difference_List.append(length_diff)    

In [69]:
%store Length_Difference_List > Length_Difference_List.txt

Writing 'Length_Difference_List' (list) to file 'Length_Difference_List.txt'.


In [51]:
Length_Difference_List = eval(open("Length_Difference_List.txt").read())  
Length_Difference_Array = np.array(Length_Difference_List)

In [52]:
Length_Difference_Array = Length_Difference_Array[:, np.newaxis]

In [88]:
Two_Features = np.concatenate((BLEU_Score_Array, Length_Difference_Array), axis=1)

In [89]:
print Two_Features.shape

(549367, 2)


In [77]:
Length_Difference_List_Test = []
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [i for i in sentence1.lower().split()]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [i for i in sentence2.lower().split()]
    length_diff = len(premise)-len(hypothesis)
    Length_Difference_List_Test.append(length_diff)    

In [78]:
%store Length_Difference_List_Test > Length_Difference_List_Test.txt

Writing 'Length_Difference_List_Test' (list) to file 'Length_Difference_List_Test.txt'.


In [81]:
Length_Difference_List_Test = eval(open("Length_Difference_List_Test.txt").read())
Length_Difference_Array_Test = np.array(Length_Difference_List_Test)

In [82]:
Length_Difference_Array_Test = Length_Difference_Array_Test[:, np.newaxis]

In [84]:
Two_Features_Test = np.concatenate((BLEU_Score_Array_Test, Length_Difference_Array_Test), axis=1)

In [85]:
print Two_Features_Test.shape

(9824, 2)


In [83]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_2 = gnb.fit(Two_Features, Target_Array).predict(Two_Features_Test)
y_pred_2_list = y_pred_2.tolist()
%store y_pred_2_list > y_pred_2_list.txt

Writing 'y_pred_2_list' (list) to file 'y_pred_2_list.txt'.


In [84]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features_Test.shape[0],(Target_Array_Test != y_pred_2).sum()))

Number of mislabeled points out of a total 9824 points : 5897


In [86]:
print 9824 - 5897

3927


In [87]:
print 3927./9824.

0.39973534202


In [88]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_2))

             precision    recall  f1-score   support

contradiction       0.42      0.21      0.28      3237
 entailment       0.39      0.59      0.47      3368
    neutral       0.40      0.40      0.40      3219

avg / total       0.41      0.40      0.38      9824



In [89]:
metrics.confusion_matrix(Target_Array_Test, y_pred_2)


array([[ 677, 1624,  936],
       [ 434, 1977,  957],
       [ 490, 1456, 1273]])

In [91]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_2_svm = clf.fit(Two_Features, Target_Array).predict(Two_Features_Test)
y_pred_list_svm_2 = y_pred_2_svm.tolist()
%store y_pred_list_svm_2 > y_pred_list_svm_2.txt

Writing 'y_pred_list_svm_2' (list) to file 'y_pred_list_svm_2.txt'.


In [92]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features_Test.shape[0],(Target_Array_Test != y_pred_2_svm).sum()))

Number of mislabeled points out of a total 9824 points : 6005


In [93]:
print 9824 - 6005

3819


In [94]:
print 3819./9824.

0.388741856678


In [95]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_2_svm))

             precision    recall  f1-score   support

contradiction       0.44      0.04      0.08      3237
 entailment       0.37      0.89      0.52      3368
    neutral       0.49      0.21      0.29      3219

avg / total       0.43      0.39      0.30      9824



In [96]:
metrics.confusion_matrix(Target_Array_Test, y_pred_2_svm)


array([[ 136, 2688,  413],
       [  71, 3013,  284],
       [ 105, 2444,  670]])

In [66]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Overlap_Percent = []
for j in range(len(pp.data)):
    sentence1 = pp.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    sentence2 = pp.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    lengtharray = np.array([len(premise), len(hypothesis)])
    length = np.min(lengtharray)
    indx = np.argmin(lengtharray)
    if length == 0:
        Overlap_Percent.append(0)
    else:
        count = 0
        if indx == 0:
            for j in range(length):
                if premise[j] in hypothesis:
                    count = count+1
        else:
            for j in range(length):
                if hypothesis[j] in premise:
                    count = count+1
        Overlap_Percent.append(count/float(length))                        

In [67]:
%store Overlap_Percent > Overlap_Percent.txt

Writing 'Overlap_Percent' (list) to file 'Overlap_Percent.txt'.


In [68]:
Overlap_Percent_Array = np.array(Overlap_Percent)

In [69]:
Overlap_Percent_Array = Overlap_Percent_Array[:, np.newaxis]

In [86]:
#nltk.download("stopwords")
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
ps = PorterStemmer()
Overlap_Percent_Test = []
for j in range(len(tt.data)):
    sentence1 = tt.data[j]['sentence1']
    sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
    premise = [ps.stem(i) for i in sentence1.lower().split() if i not in stop]
    sentence2 = tt.data[j]['sentence2']
    sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
    hypothesis = [ps.stem(i) for i in sentence2.lower().split() if i not in stop]
    lengtharray = np.array([len(premise), len(hypothesis)])
    length = np.min(lengtharray)
    indx = np.argmin(lengtharray)
    if length == 0:
        Overlap_Percent_Test.append(0)
    else:
        count = 0
        if indx == 0:
            for j in range(length):
                if premise[j] in hypothesis:
                    count = count+1
        else:
            for j in range(length):
                if hypothesis[j] in premise:
                    count = count+1
        Overlap_Percent_Test.append(count/float(length))  

In [93]:
%store Overlap_Percent_Test > Overlap_Percent_Test.txt

Writing 'Overlap_Percent_Test' (list) to file 'Overlap_Percent_Test.txt'.


In [92]:
Overlap_Percent_Array_Test = np.array(Overlap_Percent_Test)

In [94]:
Overlap_Percent_Array_Test = Overlap_Percent_Array_Test[:, np.newaxis]

In [90]:
Three_Features = np.concatenate((Two_Features, Overlap_Percent_Array), axis=1)

In [91]:
print Three_Features.shape

(549367, 3)


In [95]:
Three_Features_Test = np.concatenate((Two_Features_Test, Overlap_Percent_Array_Test), axis=1)

In [96]:
print Three_Features_Test.shape

(9824, 3)


In [97]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred_3 = gnb.fit(Three_Features, Target_Array).predict(Three_Features_Test)
y_pred_3_list = y_pred_3.tolist()
%store y_pred_3_list > y_pred_3_list.txt

Writing 'y_pred_3_list' (list) to file 'y_pred_3_list.txt'.


In [98]:
print("Number of mislabeled points out of a total %d points : %d" % (Two_Features_Test.shape[0],(Target_Array_Test != y_pred_3).sum()))

Number of mislabeled points out of a total 9824 points : 5174


In [99]:
print 9824 - 5174

4650


In [100]:
print 4650./9824.

0.473330618893


In [101]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3))

             precision    recall  f1-score   support

contradiction       0.43      0.54      0.48      3237
 entailment       0.55      0.54      0.55      3368
    neutral       0.43      0.34      0.38      3219

avg / total       0.47      0.47      0.47      9824



In [102]:
metrics.confusion_matrix(Target_Array_Test, y_pred_3)


array([[1735,  680,  822],
       [ 937, 1823,  608],
       [1340,  787, 1092]])

In [103]:
from sklearn import linear_model
clf = linear_model.SGDClassifier()
y_pred_3_svm = clf.fit(Three_Features, Target_Array).predict(Three_Features_Test)
y_pred_list_svm_3 = y_pred_3_svm.tolist()
%store y_pred_list_svm_3 > y_pred_list_svm_3.txt

Writing 'y_pred_list_svm_3' (list) to file 'y_pred_list_svm_3.txt'.


In [104]:
print("Number of mislabeled points out of a total %d points : %d" % (Three_Features_Test.shape[0],(Target_Array_Test != y_pred_3_svm).sum()))

Number of mislabeled points out of a total 9824 points : 5343


In [105]:
print 9824 - 5343

4481


In [106]:
print 4481./9824.

0.456127850163


In [107]:
from sklearn import metrics
print(metrics.classification_report(Target_Array_Test, y_pred_3_svm))

             precision    recall  f1-score   support

contradiction       0.42      0.67      0.52      3237
 entailment       0.50      0.63      0.56      3368
    neutral       0.47      0.06      0.11      3219

avg / total       0.46      0.46      0.40      9824



In [108]:
metrics.confusion_matrix(Target_Array_Test, y_pred_3_svm)


array([[2168,  938,  131],
       [1160, 2110,   98],
       [1851, 1165,  203]])